In [30]:
import pandas as pd
import numpy as np
from filegenerators import *
pd.options.display.float_format = '{:.2e}'.format

In [31]:
"""
If ic=0 => absolute sigma => dataPoint variablestd = small error
If ic!=0 rel sigma => common properties => new property value = ln((val/1.5)/(val*1.5))/4
"""
def splitSigmas(df, inputs, observables, must_be_zero):
    rel_sigmas = dict()
    abs_sigmas = dict()

    for index, row in df.iterrows():
        if row.species in inputs:
#        if row.species not in observables:
            continue
        if row.value in must_be_zero and row.species not in inputs:
            rel_sigmas[row.species] = 5e-14
        elif row.species not in inputs and row.species in observables:
            rel_sigmas[row.species] = ((row.value*1.5-row.value/2)/8)*1e-12
    return rel_sigmas

def makeBounds(df):
    bounds = dict()
    for index, row in df.iterrows():
        if row.value < 0.1:
            lb = 1e-14
            ub = 1e-13
        else:
            lb = (row.value/2)*1e-12
            ub = (row.value*1.5)*1e-12
        bounds[row.species] = [lb, ub]
    return bounds 

In [32]:
df_species_ics = pd.read_excel('../../reactions_ics_finalised.xlsx', sheet_name='Sheet7')
df_species_ics['value'] = df_species_ics['value'].astype(float)

observables = []
for index, row in df_species_ics.iterrows():
    if row.value >0:
        observables.append(row.species)
len(observables)

36

In [33]:
#inoputokat ki kell szedni
input_names =  ['nS', 'RAP', 'TG', 'dS', 'CCH', 'REF', 'Insulin', 'TG_SERCA', 'mTOR_RAP', 'casp', 'IP3R', 'Baxa', 'tBid']
#observables = ['mTORa', 'ULK', 'AMPK', 'procasp', 'Ca2ic', 'Ca2er', 'SERCA', 'AKTa', 'TSC']
#trash = ['BEC1_UVG','DAPK','DAPKa','AC_Ga','p53a','BCL2_p53','Baxa','p53_BCL2', 'MDM2', 'DRAM', 'UVG', 'BCL2', 'BCL2_PUMA']
#trash = ['BEC1_UVG','AC_Ga','p53a','BCL2_p53','Baxa','p53_BCL2','DAPKs' ,'MDM2', 'DRAM']
must_be_zero = ['casp', 'Baxa', 'tBid', 'p53a', 'PUMA']
#errors_table = list(set(observables)-set(trash))
inputs = dict()
for i in input_names:
    inputs[i] = 0.0
inputs["REF"] = 1.0
inputs["Insulin"] = 1e-10

rel = splitSigmas(df_species_ics, input_names, observables, must_be_zero)
species = df_species_ics.species.to_list()
only_vars = list(set(species)-set(input_names))
no_inp_species = []

In [34]:
#dataPoints values
columns = list(set(observables)-set(input_names))
columns.sort()
columns.insert(0,'time')
time = np.linspace(0,24,25)

dataPoints = pd.DataFrame(columns=columns)
dataPoints['time'] = time*60

#Fill in the "theoretical" stacionary conentrations
for index, row in df_species_ics.iterrows():
    if row.species in dataPoints.columns:
        if row.value == 0:
            dataPoints.loc[:,row.species] = 1e-13
        else:
            dataPoints.loc[:,row.species] = row.value*1e-12

In [35]:
#df = pd.read_excel('../../reactions_ics_finalised.xlsx', sheet_name='icranges')
df = pd.read_excel('../../reactions_ics_finalised.xlsx', sheet_name='Sheet7')
bounds = makeBounds(df)
print(bounds['mTORa'])

[1.875e-10, 5.625e-10]


In [36]:

# Directory to save files
output_directory = '20240831_stac_8adsigma'
# Create the directory if it does not exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i in range(1, 21):
    file_index = i
    generate_file(file_index, output_directory, only_vars, inputs, bounds, dataPoints, rel)
print("job finished")

job finished


In [37]:
#N_space = 24
#str_list = []
#for s in species:
#    r1 = s+(N_space-len(s))*" "+"H   1C   0    0    0G    200      6000     1000        1"
#    r2 = " 1.65326226E+00 1.00263099E-02-3.31661238E-06 5.36483138E-10-3.14696758E-14    2"
#    r3 = "-1.00095936E+04 9.90506283E+00 5.14911468E+00-1.36622009E-02 4.91453921E-05    3"
#    r4 = "-4.84246767E-08 1.66603441E-11-1.02465983E+04-4.63848842E+00 0.00000000E+00    4"
#    print(r1)
#    print(r2)
#    print(r3)
#    print(r4)